# IEMOCAP

## 01  extract transcript

In [1]:
import csv
import os
from file_util import *

In [2]:
create_folder('../data/processed/IEMOCAP')

In [3]:
out_file = '../data/processed/IEMOCAP/processed_tran.csv'
os.system('rm ' + out_file)

256

In [4]:
def extract_trans( list_in_file, out_file ) :
    
    lines = []
    
    for in_file in list_in_file:
        cnt = 0
        
        with open(in_file, 'r') as f:
            lines = f.readlines()

        with open(out_file, 'a') as f:

            csv_writer = csv.writer( f )
            lines = sorted(lines)                  # sort based on first element
            
            for line in lines:

                name = line.split(':')[0].split(' ')[0].strip()
                
                # unwanted case 
                if name[:3] != 'Ses':             # noise transcription such as reply  M: sorry
                    continue
                elif name[-3:-1] == 'XX':        # we don't have matching pair in label
                    continue
                trans = line.split(':')[1].strip()
                
                cnt += 1
                csv_writer.writerow([name, trans])

In [5]:
# [schema] ID, transcriptions [csv]

list_files = []

for x in xrange(5):
    sess_name = 'Session' + str(x+1)

    path = '../data/raw/IEMOCAP_full_release/' + sess_name + '/dialog/transcriptions/'
    file_search(path, list_files)
    list_files = sorted(list_files)

    print sess_name + ", #sum files: " + str(len(list_files))

extract_trans(list_files, out_file)

Session1, #sum files: 28
Session2, #sum files: 58
Session3, #sum files: 90
Session4, #sum files: 120
Session5, #sum files: 151


## 02 extract label

In [6]:
import csv
import os
import sys
import numpy as np
from file_util import *

In [7]:
out_file = '../data/processed/IEMOCAP/label.csv'
os.system('rm ' + out_file)

256

In [8]:
list_category = [
                'ang',
                'hap',
                'sad',
                'neu',
                'fru',
                'exc',
                'fea',
                'sur',
                'dis',
                'oth',
                'xxx'
                ]

category = {}
for c_type in list_category:
    if category.has_key(c_type):
        ;
    else:
        category[c_type] = len(category)

In [9]:
def find_category(lines):
    is_target = True
    
    id = ''
    c_label = ''
    list_ret = []
    
    for line in lines:
        
        if is_target == True:
            
            try:
                id          = line.split('\t')[1].strip()  #  extract ID
                c_label  = line.split('\t')[2].strip()  #  extract category
                if not category.has_key(c_label):
                    print "ERROR nokey" + c_label
                    sys.exit()
                
                list_ret.append( [id, c_label] )
                is_target = False

            except:
                print "ERROR " + line
                sys.exit()
        
        else:
            if line == '\n':
                is_target = True
        
    return list_ret

In [10]:
def extract_labels( list_in_file, out_file ) :
    id = ''
    lines = []
    list_ret = []
    
    for in_file in list_in_file:
        
        with open(in_file, 'r') as f:
            lines = f.readlines()
            lines = lines[2:]                           # remove head
            list_ret = find_category(lines)
            
        list_ret = sorted(list_ret)                   # sort based on first element
    
        with open(out_file, 'a') as f:
            csv_writer = csv.writer( f )
            csv_writer.writerows( list_ret )                   

In [11]:
# [schema] ID, label [csv]

list_files = []
list_avoid_dir = ['Attribute', 'Categorical', 'Self-evaluation']

for x in xrange(5):
    sess_name = 'Session' + str(x+1)

    path = '../data/raw/IEMOCAP_full_release/' + sess_name + '/dialog/EmoEvaluation/'
    file_search(path, list_files, list_avoid_dir)
    list_files = sorted(list_files)

    print sess_name + ", #sum files: " + str(len(list_files))

extract_labels(list_files, out_file)

Session1, #sum files: 28
Session2, #sum files: 58
Session3, #sum files: 90
Session4, #sum files: 120
Session5, #sum files: 151


#### 02-1 convert label category
- angry : 0        ang(1103)
- happy : 1       exc(1041), hap(595)
- sad : 2          sad(1084)   
- neutral : 3     neu(1708)

In [12]:
lines = []
with open('../data/processed/IEMOCAP/label.csv') as f :
    csv_reader = csv.reader(f)
    lines = [x for x in csv_reader]

In [13]:
with open('../data/processed/IEMOCAP/processed_label.txt', 'w') as f:
    
    with open('../data/processed/IEMOCAP/processed_ids.txt', 'w') as f2:
    
        for line in lines:
            if line[1] == 'ang':
                f.write('0\n')
                f2.write(line[0]+'\n')
            elif line[1] == 'hap':
                f.write('1\n')
                f2.write(line[0]+'\n')
            elif line[1] == 'exc':
                f.write('1\n')
                f2.write(line[0]+'\n')
            elif line[1] == 'sad':
                f.write('2\n')
                f2.write(line[0]+'\n')
            elif line[1] == 'neu':
                f.write('3\n')
                f2.write(line[0]+'\n')
            else :
                f.write('-1\n')

In [14]:
lines = []
with open('../data/processed/IEMOCAP/processed_label.txt') as f :
    lines = f.readlines()
lines = [x.strip() for x in lines]

print len([x for x in lines if x=='0'])
print len([x for x in lines if x=='1'])
print len([x for x in lines if x=='2'])
print len([x for x in lines if x=='3'])

1103
1636
1084
1708


## 03 verify

In [15]:
import csv

In [16]:
in_file = '../data/processed/IEMOCAP/processed_label.txt'
label = []
with open( in_file, 'r') as f:
    label = f.readlines()
    
label_id = [ x.strip() for x in label ]

In [17]:
in_file = '../data/processed/IEMOCAP/label.csv'
label = []
with open( in_file, 'r') as f:
    csv_reader = csv.reader( f )
    label = [x for x in csv_reader]
    
label_id = [ x[0] for x in label]

In [18]:
in_file = '../data/processed/IEMOCAP/processed_tran.csv'
tran = []
with open( in_file, 'r') as f:
    csv_reader = csv.reader( f )
    tran = [x for x in csv_reader]
    
tran_id = [ x[0] for x in tran]

In [19]:
for l, t in zip(label_id, tran_id):
    if l != t:
        print 'ERROR'

## 04 statistics

In [20]:
import numpy as np

In [21]:
in_file = '../data/processed/IEMOCAP/label.csv'
label = []
with open( in_file, 'r') as f:
    csv_reader = csv.reader( f )
    label = [x for x in csv_reader]
    
label_cat = [ x[1] for x in label]

In [22]:
count = np.zeros( len(category), dtype=np.int )

for cat in label_cat:
    count[ category[cat] ] = count[ category[cat] ] + 1

In [23]:
for key in category.keys():
    print key + '\t:' + str(count[ category[key] ])

oth	:3
ang	:1103
exc	:1041
fea	:40
xxx	:2507
sad	:1084
sur	:107
fru	:1849
hap	:595
neu	:1708
dis	:2
